In [57]:
import pandas as pd 
import numpy as np

In [58]:
df = pd.read_csv('Levels_Fyi_Salary_Data.csv') #reading in the data

In [59]:
missing_cols = [] #checking what cols have nans
for col in df.columns:
    if df[col].isnull().values.any():
        num = df[col].isnull().sum()
        missing_cols.append([col, num])

In [60]:
missing_cols

[['company', 5],
 ['level', 119],
 ['tag', 854],
 ['gender', 19540],
 ['otherdetails', 22505],
 ['dmaid', 2],
 ['Race', 40215],
 ['Education', 32272]]

In [61]:
df = df.drop(['Race', 'Education', 'otherdetails', 'level', 'tag', 'dmaid', 'cityid', 'rowNumber', 'timestamp'], axis = 1) #dropping useless cols

In [62]:
df.dropna(inplace = True) #dropping rows with nans
df.reset_index(drop=True, inplace=True) 

## Feature Engineering and more Cleaning



In [63]:
def get_state(x):
    words = x.split(', ')
    state = words[1]
    if state == 'CA':
        return 'CA'
    elif state == 'WA':
        return 'WA'
    elif state == 'NY':
        return 'NY'
    else:
        return 'Other'

In [64]:
df['state'] = df['location'].apply(get_state) #new cols abt states
df.drop(['location'], axis = 1, inplace = True)

In [65]:

top_15_popular = list(df['company'].value_counts().index)[:15]
def in_popular(x):
    if x in top_15_popular:
        return True
    else:
        return False
    
df['in_top_15'] = df['company'].apply(in_popular) #new col abt top 15 companies

In [66]:
def check_education(master, bach, doc, high, some):
    if master == 1:
        return "Masters"
    elif bach == 1:
        return "Bachelors"
    elif doc == 1:
        return "PHD"
    elif high == 1:
        return 'Highschool'
    elif some == 1:
        return "Some"
    else:
        return "Missing"

In [67]:
df['education'] = df.apply(lambda x: check_education(x.Masters_Degree, x.Bachelors_Degree, x.Doctorate_Degree, x.Highschool, x.Some_College), axis=1) #new cols abt education

In [68]:
len(df[df['education'] == 'Missing'])

15334

In [69]:
df = df[df['education'] != 'Missing']
df.reset_index(drop=True, inplace=True)
df.head()

,company,title,totalyearlycompensation,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Masters_Degree,...,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,state,in_top_15,education
0,Capital One,Data Scientist,125000,1.0,0.0,121000.0,0.0,4000.0,Male,0,...,0,0,0,0,0,0,0,Other,True,PHD
1,Bloomberg,Software Engineer,170000,0.0,0.0,142000.0,0.0,18000.0,Male,1,...,0,0,0,0,0,0,0,NY,False,Masters
2,Amazon,Data Scientist,332000,1.0,0.0,185000.0,17000.0,130000.0,Male,0,...,0,0,0,0,0,0,0,CA,True,PHD
3,Apple,Software Engineer,240000,3.0,3.0,150000.0,75000.0,15000.0,Female,0,...,0,0,0,0,0,0,0,CA,True,PHD
4,Synopsys,Software Engineer,310000,25.0,9.0,210000.0,40000.0,60000.0,Male,0,...,0,0,0,0,0,0,0,CA,False,PHD


In [70]:
def check_race(asian, white, two, black, hispanic):
    if asian == 1:
        return 'Asian'
    elif white == 1:
        return 'White'
    elif two == 1:
        return 'Two'
    elif black == 1:
        return "Black"
    elif hispanic == 1:
        return 'Hispanic'
    else:
        return 'Missing'
    
df['race'] = df.apply(lambda x: check_race(x.Race_Asian, x.Race_White, x.Race_Two_Or_More, x.Race_Black, x.Race_Hispanic), axis=1)

In [71]:
len(df[df['race'] == 'Missing'])

6175

In [72]:
df = df[df['race'] != 'Missing']
df.reset_index(drop=True, inplace=True)
df.columns

Index(['company', 'title', 'totalyearlycompensation', 'yearsofexperience',
       'yearsatcompany', 'basesalary', 'stockgrantvalue', 'bonus', 'gender',
       'Masters_Degree', 'Bachelors_Degree', 'Doctorate_Degree', 'Highschool',
       'Some_College', 'Race_Asian', 'Race_White', 'Race_Two_Or_More',
       'Race_Black', 'Race_Hispanic', 'state', 'in_top_15', 'education',
       'race'],
      dtype='object')

In [0]:
#df.to_csv('cleaned_data')

## One Hot Encoding



In [74]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [75]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['title', 'gender', 'state']),
    remainder='passthrough')

In [76]:
transformed = transformer.fit_transform(df)
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())

In [77]:
transformed_df.columns

Index(['onehotencoder__title_Business Analyst',
       'onehotencoder__title_Data Scientist',
       'onehotencoder__title_Hardware Engineer',
       'onehotencoder__title_Human Resources',
       'onehotencoder__title_Management Consultant',
       'onehotencoder__title_Marketing',
       'onehotencoder__title_Mechanical Engineer',
       'onehotencoder__title_Product Designer',
       'onehotencoder__title_Product Manager',
       'onehotencoder__title_Recruiter', 'onehotencoder__title_Sales',
       'onehotencoder__title_Software Engineer',
       'onehotencoder__title_Software Engineering Manager',
       'onehotencoder__title_Solution Architect',
       'onehotencoder__title_Technical Program Manager',
       'onehotencoder__gender_Female', 'onehotencoder__gender_Male',
       'onehotencoder__gender_Other', 'onehotencoder__state_CA',
       'onehotencoder__state_NY', 'onehotencoder__state_Other',
       'onehotencoder__state_WA', 'remainder__company',
       'remainder__totalyearl

In [78]:
cols = ['remainder__company', 'remainder__education', 'remainder__race']
transformed_df.drop(cols, axis=1, inplace=True)

In [83]:
transformed_df['remainder__in_top_15'] = transformed_df['remainder__in_top_15'].astype(int)

In [84]:
transformed_df.head()

,onehotencoder__title_Business Analyst,onehotencoder__title_Data Scientist,onehotencoder__title_Hardware Engineer,onehotencoder__title_Human Resources,onehotencoder__title_Management Consultant,onehotencoder__title_Marketing,onehotencoder__title_Mechanical Engineer,onehotencoder__title_Product Designer,onehotencoder__title_Product Manager,onehotencoder__title_Recruiter,...,remainder__Bachelors_Degree,remainder__Doctorate_Degree,remainder__Highschool,remainder__Some_College,remainder__Race_Asian,remainder__Race_White,remainder__Race_Two_Or_More,remainder__Race_Black,remainder__Race_Hispanic,remainder__in_top_15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0


## Train Test Split

In [85]:
from sklearn.model_selection import train_test_split

In [86]:
X = transformed_df.loc[:, transformed_df.columns != "remainder__totalyearlycompensation"]
y = transformed_df.remainder__totalyearlycompensation

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Machine Learning

#### Linear Regression

In [89]:
from sklearn.linear_model import LinearRegression
lin = LinearRegression()
lin.fit(X_train, y_train)

LinearRegression()

In [95]:
lin_pred = lin.predict(X_test)

In [96]:
total_squared_error = (np.sum((y_test - lin_pred)**2)) #get the sum of all the errors (error = what we want (y_test) - what we predicted (y_hat))
mean_squared_error = total_squared_error/len(y_test) #divide this by how many rows/observations we have 
print(mean_squared_error)

1248612167.2077157


In [97]:
lin.score(X_test, y_test)

0.9257397925420172

#### Ridge Regression



In [94]:
from sklearn.linear_model import Ridge
rid = Ridge()
rid.fit(X_train, y_train)

Ridge()

In [98]:
rid_pred = rid.predict(X_test)

In [99]:
total_squared_error = (np.sum((y_test - rid_pred)**2)) #get the sum of all the errors (error = what we want (y_test) - what we predicted (y_hat))
mean_squared_error = total_squared_error/len(y_test) #divide this by how many rows/observations we have 
print(mean_squared_error)

1248597912.7858734


In [100]:
rid.score(X_test, y_test)

0.925740640312327

#### SVM Regression



In [111]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [112]:
svr = SVR()
svr.fit(X_train, y_train)

SVR()

In [113]:
svr_pred = svr.predict(X_test)

In [114]:
total_squared_error = (np.sum((y_test - svr_pred)**2)) #get the sum of all the errors (error = what we want (y_test) - what we predicted (y_hat))
mean_squared_error = total_squared_error/len(y_test) #divide this by how many rows/observations we have 
print(mean_squared_error)

17039159060.458633


In [115]:
svr.score(X_test, y_test)

-0.013390322448081582

#### Stochastic Gradient Descent Regression



In [118]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor()
sgd.fit(X_train, y_train)

SGDRegressor()

In [119]:
sgd_pred = sgd.predict(X_test)

In [120]:
total_squared_error = (np.sum((y_test - sgd_pred)**2)) #get the sum of all the errors (error = what we want (y_test) - what we predicted (y_hat))
mean_squared_error = total_squared_error/len(y_test) #divide this by how many rows/observations we have 
print(mean_squared_error)

5.59088397331671e+37


In [121]:
sgd.score(X_test, y_test)

-3.3251334132077373e+27

#### KNN

In [122]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)

KNeighborsRegressor()

In [123]:
knn_pred = knn.predict(X_test)

In [124]:
total_squared_error = (np.sum((y_test - knn_pred)**2)) #get the sum of all the errors (error = what we want (y_test) - what we predicted (y_hat))
mean_squared_error = total_squared_error/len(y_test) #divide this by how many rows/observations we have 
print(mean_squared_error)

1458505490.922564


In [125]:
knn.score(X_test, y_test)

0.9132565554148582

#### Decision Tree 

In [130]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth = 2)
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2)

In [131]:
dt_pred = dt.predict(X_test)

In [132]:
total_squared_error = (np.sum((y_test - dt_pred)**2)) #get the sum of all the errors (error = what we want (y_test) - what we predicted (y_hat))
mean_squared_error = total_squared_error/len(y_test) #divide this by how many rows/observations we have 
print(mean_squared_error)

5415917096.987468


In [133]:
dt.score(X_test, y_test)

0.6778926733535366

## Graphs



In [77]:
import plotly.express as px

In [78]:
fig = px.histogram(df, x="yearsofexperience", title = 'Distribution of Years of Experience in STEM')
fig.show()

In [79]:
fig = px.histogram(df, x="basesalary", title = 'Distribution of Base Salaries in STEM')
fig.show()

In [80]:
fig = px.histogram(df, x="totalyearlycompensation", title = 'Distribution of Total Yearly Compensation in STEM', color = 'in_top_15')
fig.show()

In [89]:
companys = df.groupby(by = 'company').count().reset_index().sort_values(by = 'title', ascending = False).head(10)

In [82]:
fig = px.pie(companys, values='title', names='company', title='Top 10 Most Popular Tech Companies')
fig.show()

In [83]:
gender_counts = df.groupby(by = 'gender')['company'].count().reset_index().head(3)
fig = px.pie(gender_counts, values='company', names='gender', title='Gender Breakdown in STEM')
fig.show()

In [84]:
top_pay = df.groupby(by = 'company')['totalyearlycompensation'].mean().reset_index().sort_values(by = 'totalyearlycompensation', ascending = False).head(30)

In [85]:
fig = px.bar(top_pay, x='company', y='totalyearlycompensation', title = 'Top 30 STEM Companies with the Highest Average Yearly Pay')
fig.show()

In [86]:
sw = df[df['title'] == 'Software Engineer']
fig = px.histogram(sw, x="totalyearlycompensation", title = 'Distribution of Software Engineers Total Yearly Compensation in STEM', color = 'state')
fig.show()

In [87]:
ds = df[df['title'] == 'Data Scientist']
fig = px.histogram(ds, x="totalyearlycompensation", title = 'Distribution of Data Scientists Total Yearly Compensation in STEM', color = 'state')
fig.show()

In [88]:
#bryce simon - gender vs totalyearconsumption - scatterplots
#cassidy max - totalyearlycompensation - legend with in top15 - histogram
#aadya anaya - job positions in stem - pie chart
#bryce simon - job titles and their base salarys - barchart
#four groups - swe basesalary - histogram - states as legend

## Advanced Features



In [0]:
#company, title, location, race, gender, education, experience so far